In [1]:
import pandas as pd

In [ ]:
# read the excel file and see the sheet names
xl_files = pd.ExcelFile(r'..\landing_zone\2026\Hyundia\2025-11-1 HACC MAF - 20251113.xlsx')

In [17]:
target_sheets = "Master"
if target_sheets in xl_files.sheet_names:
    hyundai_master_sheet_df = pd.read_excel(r"..\landing_zone\2026\Hyundia\2025-11-1 HACC MAF - 20251113.xlsx", sheet_name=target_sheets, skiprows=1)


In [22]:
hyundai_master_sheet_df.columns

Index(['Model \nYear \nFrom', 'Model \nYear \nTo', 'Model', 'Trim 1', 'Trim 2',
       'Trim 3', 'Trim 4', 'Trim 5', 'Trim 6', 'Trim 7', 'Trim 8',
       'Accessory \nCategory (EN)', 'Accessory Group (EN)',
       'Accessory Description (EN)', 'Accessory \nCategory (FR)',
       'Accessory Group (FR)', 'Accessory Description (FR)', 'Part\nNumber',
       'DNET', 'MSRP', 'Labour Rate', 'Suggested Labour Hours',
       'Suggested Base Retail Price w/labour', 'Yearly Rate', 'Weekly Rate',
       '#of Payments', 'Weekly Payment', 'PKG Code', 'Comments (EN)',
       'Comments (FR)', 'New Part SOS', 'Outgoing Part Flag', 'MAF Update'],
      dtype='object')

In [34]:
hyundai_master_sheet_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4313 entries, 0 to 4312
Data columns (total 33 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Model 
Year 
From                     4313 non-null   int64  
 1   Model 
Year 
To                       4313 non-null   int64  
 2   Model                                 4313 non-null   object 
 3   Trim 1                                3845 non-null   object 
 4   Trim 2                                3505 non-null   object 
 5   Trim 3                                2922 non-null   object 
 6   Trim 4                                1744 non-null   object 
 7   Trim 5                                1283 non-null   object 
 8   Trim 6                                708 non-null    object 
 9   Trim 7                                99 non-null     object 
 10  Trim 8                                0 non-null      float64
 11  Accessory 
Catego

In [35]:
hyundai_master_sheet_df = hyundai_master_sheet_df.dropna(axis=1, how='all')

In [36]:
hyundai_master_sheet_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4313 entries, 0 to 4312
Data columns (total 32 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Model 
Year 
From                     4313 non-null   int64  
 1   Model 
Year 
To                       4313 non-null   int64  
 2   Model                                 4313 non-null   object 
 3   Trim 1                                3845 non-null   object 
 4   Trim 2                                3505 non-null   object 
 5   Trim 3                                2922 non-null   object 
 6   Trim 4                                1744 non-null   object 
 7   Trim 5                                1283 non-null   object 
 8   Trim 6                                708 non-null    object 
 9   Trim 7                                99 non-null     object 
 10  Accessory 
Category (EN)              4313 non-null   object 
 11  Accessory Group (

In [ ]:
key_columns_to_keep = ['Model \nYear \nFrom', 'Model', 'Trim 1', 'Trim 2',
       'Trim 3', 'Trim 4', 'Trim 5', 'Trim 6', 'Trim 7', 'Trim 8',
       'Accessory Description (EN)', 'Accessory \nCategory (FR)',
       'Part\nNumber', 'DNET', 'MSRP', 'Labour Rate', 'Suggested Labour Hours',
       'Suggested Base Retail Price w/labour', 'Yearly Rate', 'Weekly Rate',
       '#of Payments', 'Weekly Payment', 'PKG Code', 'Comments (EN)',
       'Comments (FR)', 'New Part SOS', 'Outgoing Part Flag', 'MAF Update']

In [39]:
hyundai_master_sheet_df["Labour Rate"].value_counts()

Labour Rate
136.95    4313
Name: count, dtype: int64

In [43]:
def show_unique_values(df, columns):
    """
    Display unique values for the specified columns in a DataFrame.
    
    Parameters:
    df (pd.DataFrame): The DataFrame to analyze.
    columns (list): List of column names to check.
    
    Returns:
    dict: A dictionary with column names as keys and unique values as lists.
    """
    unique_dict = {}
    for col in columns:
        if col in df.columns:
            unique_dict[col] = df[col].unique().tolist()
            print(f"{col}: {df[col].unique().tolist()}")
        else:
            unique_dict[col] = "Column not found"
    return unique_dict


result = show_unique_values(hyundai_master_sheet_df, ['Trim 1', 'Trim 2',
       'Trim 3', 'Trim 4', 'Trim 5', 'Trim 6', 'Trim 7'])


Trim 1: ['Pref', nan, 'Ess', 'Advanced', '2.5T Advanced', 'Prestige', '2.0T Advanced', '2.5T Adv', '3.5T e-SC', 'Pref HEV', 'N', '2.5T Advanced 5P', 'Pref*', '3.5T Coupe', 'Pref SR', '3.5T e-SC Prestige', 'Pref Trend', 'Pref LR']
Trim 2: ['Urban', nan, 'Trend', 'Pref', 'Ult', 'Lux', 'Performance', '2.5T Advanced Plus', '2.5T Prestige', '2.0T Prestige', '3.5T Sport Plus', '3.5T Advanced', 'Trend AWD', 'Pref HEV w/ Trend', '2.5T Advanced Tech Pkg 5P', 'Trend*', 'XRT', 'Pref LR', '2.5T Advanced Tech', 'Prestige', '3.5T e-SC Prestige Black', '3.5T Prestige Black']
Trim 3: ['Calli', nan, 'N-Line', 'Ult', 'Lux', 'Trend', '2.5T Prestige', '3.3T Advanced', '3.5T Prestige', 'XRT', '3.5T Advanced Tech Pkg 7P', '3.5T Sport', '3.5T Advanced Tech', '3.3T Sport', 'Calli ICE', 'Lux HEV']
Trim 4: [nan, 'Lux', 'N-Line', '3.5T Sport', '3.3T Sport', '3.5T Prestige 7P', 'Sport', 'N-Line Ult', 'Ult Calli', '3.5T Prestige', 'Lux HEV', 'HEV', 'Calli HEV']
Trim 5: [nan, 'Ult HEV', 'HEV', 'N-Line Ult', '3.5T S

In [44]:
get_unique_models = hyundai_master_sheet_df["Model"].unique()

In [45]:
get_unique_models

array(['Palisade', 'Tucson', 'Santa Cruz', 'Elantra', 'Ioniq 6', 'Kona',
       'Kona EV', 'Venue', 'Ioniq 5', 'GV60', 'GV70', 'GV70 EV', 'G70',
       'G80 EV', 'G80', 'G90', 'GV80', 'Sonata', 'Santa Fe', 'GV80 Coupe',
       'Ioniq 9'], dtype=object)

In [66]:
Elantra_df = hyundai_master_sheet_df[hyundai_master_sheet_df["Model"]== "Elantra"]

In [ ]:
# drop empty columns 
Elantra_df = Elantra_df.dropna(axis=1, how='all')


,Model \nYear \nFrom,Model \nYear \nTo,Model,Trim 1,Trim 2,Trim 3,Trim 4,Trim 5,Trim 6,Accessory \nCategory (EN),...,Suggested Base Retail Price w/labour,Yearly Rate,Weekly Rate,#of Payments,Weekly Payment,Comments (EN),Comments (FR),New Part SOS,Outgoing Part Flag,MAF Update
306,2024,2024,Elantra,Ess,Pref,Lux,N-Line,HEV,N,Interior,...,284.190,0.0599,0.001152,365.26,0.953617,NaN,NaN,NaN,NaN,NaN
307,2024,2024,Elantra,Ess,Pref,Lux,N-Line,HEV,N,Interior,...,107.910,0.0599,0.001152,365.26,0.362099,NaN,NaN,NaN,NaN,NaN
308,2024,2024,Elantra,Ess,Pref,Lux,N-Line,HEV,N,Interior,...,56.325,0.0599,0.001152,365.26,0.189002,Installation required,Installation requise,NaN,NaN,NaN
309,2024,2024,Elantra,Ess,Pref,Lux,N-Line,HEV,N,Interior,...,78.260,0.0599,0.001152,365.26,0.262606,NaN,NaN,NaN,NaN,NaN
310,2024,2024,Elantra,Ess,Pref,Lux,N-Line,HEV,N,Interior,...,258.120,0.0599,0.001152,365.26,0.866138,Remove carpet mats prior to installation,Veuillez retirer les tapis protecteurs d'origi...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3460,2026,2026,Elantra,Ess,Pref,Lux,NaN,NaN,NaN,Exterior,...,8.140,0.0599,0.001152,365.26,0.027314,5 required per wheel (or 4 plus 000AM-82617),5 requis par roue (ou 4 plus 000AM-82617),NaN,NaN,NaN
3461,2026,2026,Elantra,NaN,Pref,Lux,N-Line,HEV,N,Exterior,...,95.715,0.0599,0.001152,365.26,0.321178,To be used with alloy wheels,Pour utilisation avec jantes en alliage,NaN,NaN,NaN
3462,2026,2026,Elantra,Ess,NaN,NaN,NaN,NaN,NaN,Exterior,...,101.105,0.0599,0.001152,365.26,0.339264,To be used with steel wheels with caps,Pour utilisation avec jantes en acier avec cap...,NaN,NaN,NaN
3463,2026,2026,Elantra,Ess,Pref,Lux,NaN,NaN,NaN,Exterior,...,90.955,0.0599,0.001152,365.26,0.305205,To be used with alloy wheels,Pour utilisation avec jantes en alliage,NaN,NaN,NaN


In [69]:
# identify all the trim columns

trim_colms = [col for col in Elantra_df.columns if "Trim" in col]

In [85]:
trim_colms

['Trim 1', 'Trim 2', 'Trim 3', 'Trim 4', 'Trim 5', 'Trim 6']

In [83]:
# go through the Elantra_df, look through all the trim columns, and get unique values in each column and save them to a set 

model_trims_set = []

for col in trim_colms:
    
    model_trims_set.extend(Elantra_df[col].dropna().unique().tolist())
        
print(model_trims_set)



['Ess', 'Pref', 'Lux', 'N-Line', 'HEV', 'N']


In [95]:
Elantra_df[Elantra_df[trim_colms[0]]=="Ess"].info()

<class 'pandas.core.frame.DataFrame'>
Index: 228 entries, 306 to 3464
Data columns (total 30 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Model 
Year 
From                     228 non-null    int64  
 1   Model 
Year 
To                       228 non-null    int64  
 2   Model                                 228 non-null    object 
 3   Trim 1                                228 non-null    object 
 4   Trim 2                                225 non-null    object 
 5   Trim 3                                225 non-null    object 
 6   Trim 4                                162 non-null    object 
 7   Trim 5                                171 non-null    object 
 8   Trim 6                                153 non-null    object 
 9   Accessory 
Category (EN)              228 non-null    object 
 10  Accessory Group (EN)                  228 non-null    object 
 11  Accessory Description

In [ ]:
for col_id in range(0, len(trim_colms)):
    
    Elantra_df[trim_colms[col_id]]



0
1
2
3
4
5


In [ ]:
def assign_trims_respective_models(Palisade_df, ):
    for index_ in range (0, trim_colms):
        
